In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

In [3]:
# get links in 9 way dataset
with open('9_way_list-Copy1.txt', 'r') as file:
    composers_9 = file.readlines()
    
def invert_string(input_string):
    # Split the input string by comma
    substrings = input_string.split(",")
    
    # Ensure that there are exactly two substrings
    if len(substrings) != 2:
        return "Invalid input format. Please provide a string in the format 'A, B'."
    
    # Swap the positions of the substrings and concatenate with underscore
    inverted_string = substrings[1] + "_" + substrings[0]
    
    return inverted_string

composers_9 = [invert_string(composer[:-1])[1:] for composer in composers_9]

composers_9 = ['https://imslp.org/wiki/List_of_works_by_' + composer for composer in composers_9]
print(composers_9)

['https://imslp.org/wiki/List_of_works_by_Johann_Sebastian_Bach', 'https://imslp.org/wiki/List_of_works_by_Ludwig_van_Beethoven', 'https://imslp.org/wiki/List_of_works_by_Fr%C3%A9d%C3%A9ric_Chopin', 'https://imslp.org/wiki/List_of_works_by_Joseph_Haydn', 'https://imslp.org/wiki/List_of_works_by_Franz_Liszt', 'https://imslp.org/wiki/List_of_works_by_Wolfgang_Amadeus_Mozart', 'https://imslp.org/wiki/List_of_works_by_Franz_Schubert', 'https://imslp.org/wiki/List_of_works_by_Robert_Schumann', 'https://imslp.org/wiki/List_of_works_by_Aleksand_Scriabin']


In [64]:
# get links in 100 way dataset
with open('100_way_list-Copy1.txt', 'r') as file:
    composers_100 = file.readlines()

def invert_string(input_string):
    # Split the input string by comma
    substrings = input_string.split(",")
    
    # Ensure that there are exactly two substrings
    if len(substrings) != 2:
        return "Invalid input format. Please provide a string in the format 'A, B'."
    
    # Swap the positions of the substrings and concatenate with underscore
    inverted_string = substrings[1] + "_" + substrings[0]
    
    return inverted_string

composers_100_new = [invert_string(composer[:-1])[1:] for composer in composers_100]

composers_100 = [composer[:-1] for composer in composers_100]
composers_100 = ["https://imslp.org/wiki/Category:" + composer for composer in composers_100]

composers_100_new = ['https://imslp.org/wiki/List_of_works_by_' + composer for composer in composers_100_new]
print(composers_100)
print(composers_100_new)

['https://imslp.org/wiki/Category:Czerny,_Carl', 'https://imslp.org/wiki/Category:Liszt,_Franz', 'https://imslp.org/wiki/Category:Bach,_Johann_Sebastian', 'https://imslp.org/wiki/Category:Heller,_Stephen', 'https://imslp.org/wiki/Category:Beethoven,_Ludwig_van', 'https://imslp.org/wiki/Category:Kalkbrenner,_Friedrich_Wilhelm', 'https://imslp.org/wiki/Category:Schumann,_Robert', 'https://imslp.org/wiki/Category:Schubert,_Franz', 'https://imslp.org/wiki/Category:Chopin,_Fr%C3%A9d%C3%A9ric', 'https://imslp.org/wiki/Category:Moszkowski,_Moritz', 'https://imslp.org/wiki/Category:Reger,_Max', 'https://imslp.org/wiki/Category:Clementi,_Muzio', 'https://imslp.org/wiki/Category:Kirchner,_Theodor', 'https://imslp.org/wiki/Category:Rubinstein,_Anton', 'https://imslp.org/wiki/Category:Smith,_Sydney', 'https://imslp.org/wiki/Category:Mozart,_Wolfgang_Amadeus', 'https://imslp.org/wiki/Category:Raff,_Joachim', 'https://imslp.org/wiki/Category:Alkan,_Charles-Valentin', 'https://imslp.org/wiki/Category

In [14]:
# code the previous team uses to find all the urls for a composer

def find_score_url(url):
    """
    This function takes in one arguments:
    1) url, a composer url page to parse
    This function returns a list of piece url for a specific composer.
    """
    list_page = [url]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    # add everything on the first page to the list
    result = soup.find_all('div', class_="jq-ui-tabs" )
    if result == []:
        return "badlink"
    scores = result[0].find_all('a',class_='categorypagelink')
    url_list = [link.get('href') for link in scores]
    # check sub-page
    list_page = find_next_score_url(url)
    if list_page != []:
        for page in list_page:
            r = requests.get(page)
            soup = BeautifulSoup(r.text, "html.parser")
            url_list += [link.get('href') for link in soup.find_all('a',class_='categorypagelink')]
    return url_list

def find_next_score_url(url):
    """
    Helper function for find_score_url
    """
    result = []
    while True:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        url_list = [link.get('href') for link in soup.find_all('a',class_='categorypaginglink',string='next 200')]
        if url_list == []:
            break
        url = 'http://imslp.org' + url_list[-1]
        result.append(url)
    return result

In [65]:
def get_hyperlinks(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all anchor tags (hyperlinks) in the page
        links = soup.find_all('a')
        
        # Extract the href attribute from each anchor tag
        hyperlinks = []
        for link in links:
            href = link.get('href')
            # Check if the href starts with "/wiki/"
            if href and href.startswith("/wiki/"):
                # Add the header and append to the list of hyperlinks
                hyperlinks.append(urljoin("https://imslp.org", href))
        
        return hyperlinks
    else:
        # If the request was not successful, print an error message
        print("Failed to retrieve the page:", response.status_code, url)
        return []

In [69]:
def get_hyperlinks_multiple_with_replacement(composer_list, other_composer_list):
    all_hyperlinks = []
    for i in range(len(composer_list)):
        # Send a GET request to the URL
        url = composer_list[i]
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all anchor tags (hyperlinks) in the page
            links = soup.find_all('a')
        
            # Extract the href attribute from each anchor tag
            hyperlinks = []
            for link in links:
                href = link.get('href')
                # Check if the href starts with "/wiki/"
                if href and href.startswith("/wiki/"):
                    # Add the header and append to the list of hyperlinks
                    hyperlinks.append(urljoin("https://imslp.org", href))
            
            all_hyperlinks+=hyperlinks
            
        else:
            try:
                url = other_composer_list[i]
                all_hyperlinks+=hyperlinks
            except:
                # If the request was not successful, print an error message
                print("Failed to retrieve the page:", response.status_code, url)
    return all_hyperlinks

In [26]:
urls_9 = []
for composer in composers_9:
    urls_9+= get_hyperlinks(composer)
urls_9 = list(set(urls_9))

Failed to retrieve the page: 404
['https://imslp.org/wiki/Main_Page', 'https://imslp.org/wiki/List_of_works_by_Johann_Sebastian_Bach', 'https://imslp.org/wiki/Special:WhatLinksHere/List_of_works_by_Johann_Sebastian_Bach', 'https://imslp.org/wiki/Special:RecentChangesLinked/List_of_works_by_Johann_Sebastian_Bach', 'https://imslp.org/wiki/Main_Page', 'https://imslp.org/wiki/Special:RecentChanges', 'https://imslp.org/wiki/Special:Random', 'https://imslp.org/wiki/IMSLP:Difficulty', 'https://imslp.org/wiki/Category:Composers', 'https://imslp.org/wiki/Category:People', 'https://imslp.org/wiki/Category:People_by_nationality', 'https://imslp.org/wiki/Browse_people_by_time_period', 'https://imslp.org/wiki/IMSLP:View_Genres', 'https://imslp.org/wiki/Category:WIMA_files', 'https://imslp.org/wiki/Category:Performers', 'https://imslp.org/wiki/Category:Pages_with_commercial_recordings', 'https://imslp.org/wiki/IMSLP:Contributor_Portal', 'https://imslp.org/wiki/IMSLP:IRC_Access', 'https://imslp.org/w

In [70]:
urls_100 = get_hyperlinks_multiple_with_replacement(composers_100_new, composers_100)

In [72]:
urls_100 = list(set(urls_100))

['https://imslp.org/wiki/Liebster_Jesu,_wir_sind_hier,_BWV_730_(Bach,_Johann_Sebastian)', 'https://imslp.org/wiki/Tarantelle,_Op.6_(Saint-Sa%C3%ABns,_Camille)', 'https://imslp.org/wiki/4_Morceaux,_Op.30_(Arensky,_Anton)', 'https://imslp.org/wiki/Prelude,_HWV_570_(Handel,_George_Frideric)', 'https://imslp.org/wiki/Piano_Sonata_No.11,_Op.730_(Czerny,_Carl)', 'https://imslp.org/wiki/Duet_in_D_major,_D.Anh.I/6_(Schubert,_Franz)', 'https://imslp.org/wiki/Category:Busoni,_Ferruccio', 'https://imslp.org/wiki/2_Offertories,_Op.65_(Faur%C3%A9,_Gabriel)', 'https://imslp.org/wiki/Matth%C3%A4us-Passion,_H.782_(Bach,_Carl_Philipp_Emanuel)', 'https://imslp.org/wiki/Triumphal_Overture_on_Russian_Themes,_Op.7_(Lyapunov,_Sergey)', 'https://imslp.org/wiki/Concertino_in_E_minor,_Op.45_(Weber,_Carl_Maria_von)', 'https://imslp.org/wiki/The_Tale_of_Tsar_Saltan_(opera)_(Rimsky-Korsakov,_Nikolay)', 'https://imslp.org/wiki/2_Nocturnes,_Op.38_(Jensen,_Adolf)', 'https://imslp.org/wiki/Salve_Regina,_D.223_(Schube

In [74]:
def save_list_to_file(lst, filename):
    with open(filename, 'w') as f:
        for item in lst:
            f.write(str(item) + '\n')

In [ ]:
# create file for all urls from 9-way composers
save_list_to_file(urls_9, '9_way_urls_for_all_scores.txt')

In [75]:
# create file for all urls from 100-way composers
save_list_to_file(urls_100, '100_way_urls_for_all_scores.txt')